In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
0,0,Duku,NG,2020-04-20 00:14:38,11.24,4.91,88.56,43,63,10.07,broken clouds,0.00,0.0
1,1,Jalu,LY,2020-04-20 00:14:38,29.03,21.55,71.15,24,38,8.03,scattered clouds,0.00,0.0
2,2,Yamada,JP,2020-04-20 00:14:06,39.47,141.95,47.59,89,100,9.35,light rain,1.17,0.0
3,3,Ushuaia,AR,2020-04-20 00:13:00,-54.80,-68.30,41.00,86,90,27.51,overcast clouds,0.00,0.0
4,4,Kavaratti,IN,2020-04-20 00:14:39,10.57,72.64,85.08,69,42,5.19,scattered clouds,0.00,0.0


In [27]:
# Ask the customer to add a minimum, maximum temperature value, and if he'she likes to have rain/snow:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_input = str(input("Do you want it to be raining? (yes/no) "))
snow_input = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [28]:
# Convert customer input:
if rain_input.lower() == "yes":
    with_rain = city_data_df['Rain inches (last 3 hours)'] > 0
else:
    with_rain = city_data_df['Rain inches (last 3 hours)'] == 0
    
if snow_input.lower() == "yes":
    with_snow = city_data_df['Snow inches (last 3 hours)'] > 0
else:
    with_snow = city_data_df['Snow inches (last 3 hours)'] == 0
    
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       with_rain & with_snow
                                      ]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
1,1,Jalu,LY,2020-04-20 00:14:38,29.03,21.55,71.15,24,38,8.03,scattered clouds,0.0,0.0
13,13,San Rafael,AR,2020-04-20 00:14:25,-34.62,-68.33,66.72,34,1,5.08,clear sky,0.0,0.0
32,32,Carnarvon,AU,2020-04-20 00:14:46,-24.87,113.63,71.60,83,49,11.41,scattered clouds,0.0,0.0
44,44,Saint George,US,2020-04-20 00:11:56,37.10,-113.58,71.60,26,20,11.41,few clouds,0.0,0.0
63,63,Arraial Do Cabo,BR,2020-04-20 00:14:55,-22.97,-42.03,73.67,70,96,4.79,overcast clouds,0.0,0.0
70,70,Port Alfred,ZA,2020-04-20 00:14:57,-33.59,26.89,68.00,86,14,7.36,few clouds,0.0,0.0
78,78,Pampa,US,2020-04-20 00:13:45,35.54,-100.96,73.40,28,1,7.31,clear sky,0.0,0.0
96,96,Byron Bay,AU,2020-04-20 00:15:04,-28.65,153.62,72.00,56,1,8.05,clear sky,0.0,0.0
97,97,Caloundra,AU,2020-04-20 00:12:25,-26.80,153.13,75.00,60,0,3.00,clear sky,0.0,0.0
99,99,Awjilah,LY,2020-04-20 00:15:05,29.11,21.29,71.08,24,32,8.59,scattered clouds,0.0,0.0


In [29]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jalu,LY,71.15,scattered clouds,29.03,21.55,
13,San Rafael,AR,66.72,clear sky,-34.62,-68.33,
32,Carnarvon,AU,71.60,scattered clouds,-24.87,113.63,
44,Saint George,US,71.60,few clouds,37.10,-113.58,
63,Arraial Do Cabo,BR,73.67,overcast clouds,-22.97,-42.03,
...,...,...,...,...,...,...,...
544,Yangambi,CD,70.32,overcast clouds,0.81,24.43,
547,Inongo,CD,70.61,overcast clouds,-1.95,18.27,
550,Bonham,US,66.99,fog,33.58,-96.18,
560,San Ramon,US,68.00,clear sky,37.78,-121.98,


In [6]:
# Retrieve hotels from nearby search on Google:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [30]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
        
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jalu,LY,71.15,scattered clouds,29.03,21.55,Jalu Hotel
13,San Rafael,AR,66.72,clear sky,-34.62,-68.33,Hotel San Rafael
32,Carnarvon,AU,71.60,scattered clouds,-24.87,113.63,Hospitality Carnarvon
44,Saint George,US,71.60,few clouds,37.10,-113.58,Best Western Plus Abbey Inn
63,Arraial Do Cabo,BR,73.67,overcast clouds,-22.97,-42.03,Pousada Porto Praia
...,...,...,...,...,...,...,...
544,Yangambi,CD,70.32,overcast clouds,0.81,24.43,
547,Inongo,CD,70.61,overcast clouds,-1.95,18.27,Hotel Du Lac
550,Bonham,US,66.99,fog,33.58,-96.18,Economy Inn
560,San Ramon,US,68.00,clear sky,37.78,-121.98,San Ramon Marriott


In [33]:
# Remove blank hotel
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ""]
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jalu,LY,71.15,scattered clouds,29.03,21.55,Jalu Hotel
13,San Rafael,AR,66.72,clear sky,-34.62,-68.33,Hotel San Rafael
32,Carnarvon,AU,71.60,scattered clouds,-24.87,113.63,Hospitality Carnarvon
44,Saint George,US,71.60,few clouds,37.10,-113.58,Best Western Plus Abbey Inn
63,Arraial Do Cabo,BR,73.67,overcast clouds,-22.97,-42.03,Pousada Porto Praia
...,...,...,...,...,...,...,...
536,Hualmay,PE,70.61,few clouds,-11.10,-77.61,Hotel Restaurant Turístico Casablanca
547,Inongo,CD,70.61,overcast clouds,-1.95,18.27,Hotel Du Lac
550,Bonham,US,66.99,fog,33.58,-96.18,Economy Inn
560,San Ramon,US,68.00,clear sky,37.78,-121.98,San Ramon Marriott


In [39]:
# Save and upload the new DataFrame as WeatherPy_vacation.csv.
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# Export the hotel_df into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# Create pop-up markers:
# HTML code:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [41]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [42]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [43]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# heat layer for temp is not required:
    # heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
    # fig.add_layer(heat_layer)

# add pop-up marker:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))